In [64]:
from io import BytesIO
from kfp.components import load_component
from os.path import join as path
from requests import get as GET
import tarfile
from tarfile import TarError
from urllib.parse import urlparse
import zipfile

In [65]:
url = 'gs://ml-pipeline-playground/coin.tar.gz'
http = 'https://storage.googleapis.com/ml-pipeline-playground/coin.tar.gz'
dir = '/Users/ryan/c/pipelines/backend'
tar_gz = path(dir, 'coin.tar.gz')
tar = path(dir, 'coin.tar')
zip = path(dir, 'coin.zip')
yaml = path(dir, 'pipeline.yaml')

In [117]:
file = '/Users/ryan/c/pipelines/samples/core/condition/condition.py.zip'

In [66]:
resp = GET(http); resp

<Response [200]>

In [67]:
resp.content

b'\x1f\x8b\x08\x08\xc3\xbc\xe4[\x02\xffcondition.py.tar\x00\xed\x18\xbbn\xdb0\xd0\xb3\xbf\x82\xc8\xe2\x14\xa8d=\xea\x18U\x90\xa1K\x97\x02]\n\xb4C\xd1\x81\xa1\x18\x9b\xb1D\xb2$\x95\xc0M\xf3\xef=K\xb2L\xc5\x92\xe0$\x8e\x9b\x87\xce\x83%\xf2xo\xdd\x83\x92I\x9a0N\xdd%N\x93\xc1\xd3\x80\x07p\xe2y\x03\xdf\xf3\xc2\x89\xe7\x0f\xbc`2\x9d\x04a\xbe\xeey\xfe\xf4\x83\x1f\x0e\xfc0\x9c\xfa~x\x12z\xb0\xeeO\xa6\xd3\xc9\x00y\x83\x03@\xa6\rV\x08\r~\xb3\x94\xf1\xd9e+\x9eT,\xc5j9S"\x93mJ\x02T\xff/\x04\xb0d\xdf\xa9\xd2L\xf0\x08a5\x13R\x89K\x97\x89\xf1\x95\x8f\x139\xc7\xfep\xc1x\x1c\xa1\x1fB-.\x12q=L\xa9\xc1168\x1a"4\xa3\x9c*l\xe8W\x9c\xd2\x08\xc92\x92\x9c\x8b\x84I\x87\x08\xc6\x9d\xa1\x96\x94\xac0\x81r\x96Rn\xf4\xea\x05!\x89\x15\x1c1\xc07B?\x7f\xc1\x12l\xa9\xa5\x84#\xa6\x89\x0e h\xaa\xae\x18\xa1\x9f\x08\x11\x197w\x18\xaa\x8c\x83 \x80eh*\x13\x10(g\xe3 \xc6e\xb6fYcZ\xae8\x88\xe7\x84rF"3\x80\x9d\xef\x14\xabD\xf0\x98\x190\x8c\xe3\xe7\xab\xdaPY\x1eu\xe0wG\xa56\x1e]|\xd6p\x85\x93\x0c\xb6G77\x85\xc4\xee\x86\x8ck\x9d\

In [68]:
resp.ok

True

In [98]:
def url_or_file_to_bytes(url):
  parsed = urlparse(url)
  if not parsed.scheme:
    with open(url, 'rb') as f:
        return f.read()
  else:
    resp = GET(url)
    assert resp.ok
    return resp.content

In [86]:
def try_extract_pipeline_tar(bytes):
    try:
        with tarfile.open(fileobj=BytesIO(bytes), mode='r') as f:
            names = f.getnames()
            if names == ['pipeline.yaml']:
                tar_info = f.getmember('pipeline.yaml')
                if tar_info.isfile():
                    return f.extractfile(tar_info).read()
                raise Exception('"pipeline.yaml" in TAR archive is not a regular file')
            raise Exception('Expected TAR archive to contain only a "pipeline.yaml"; found %s' % names)
    except TarError:
        return None       

In [93]:
from zipfile import BadZipFile, ZipFile

In [94]:
def try_extract_pipeline_zip(bytes):
    try:
        with ZipFile(BytesIO(bytes), mode='r') as f:
            names = f.namelist()
            if names == ['pipeline.yaml']:
                return f.read('pipeline.yaml')
            raise Exception('Expected ZIP archive to contain only a "pipeline.yaml"; found %s' % names)
    except BadZipFile:
        return None            

In [105]:
from kfp import components

In [108]:
from kfp.components._yaml_utils import load_yaml

In [106]:
components.load_yaml

AttributeError: module 'kfp.components' has no attribute 'load_yaml'

In [120]:
module_url = 'https://raw.githubusercontent.com/kubeflow/pipelines/0.1.31/samples/core/condition/condition.py'
eval()

SyntaxError: invalid syntax (<string>, line 16)

In [127]:
module_src = GET(module_url).content.decode(); module_src

'#!/usr/bin/env python3\n# Copyright 2019 Google LLC\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#      http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n\nimport kfp\nfrom kfp import dsl\n\n\ndef random_num_op(low, high):\n    """Generate a random number between low and high."""\n    return dsl.ContainerOp(\n        name=\'Generate random number\',\n        image=\'python:alpine3.6\',\n        command=[\'sh\', \'-c\'],\n        arguments=[\'python -c "import random; print(random.randint($0, $1))" | tee $2\', str(low), str(high), \'/tm

In [130]:
from os.path import basename
import sys
from tempfile import mkdtemp

In [131]:
tmp = mkdtemp()

In [143]:
module_name = basename(module_url)

In [134]:
with open(path(tmp, basename(module_url)), 'w') as f:
    f.write(module_src)

In [136]:
with open(path(tmp, '__init__.py'), 'w') as f:
    f.write('')

In [137]:
sys.path.append(tmp)
from condition import flipcoin_pipeline
sys.path.pop()

In [147]:
pipeline = path(tmp, module_name + '.zip')

In [139]:
from kfp.compiler import Compiler

In [140]:
compiler = Compiler()
compiler.compile(flipcoin_pipeline, pipeline)

In [146]:
tmp

'/var/folders/m0/mj2x82p1527349z6mn8btgtr0000gr/T/tmp8uhbrus1'

In [118]:
from kfp.components import load_component_from_text, load_component

In [113]:
txt = try_extract_pipeline_tar(bytes); txt

b'apiVersion: argoproj.io/v1alpha1\nkind: Workflow\nmetadata:\n  generateName: pipeline-flip-coin-\nspec:\n  arguments:\n    parameters: []\n  entrypoint: pipeline-flip-coin\n  serviceAccountName: pipeline-runner\n  templates:\n  - inputs:\n      parameters:\n      - name: flip-output\n    name: condition-1\n    steps:\n    - - arguments:\n          parameters:\n          - name: flip-output\n            value: \'{{inputs.parameters.flip-output}}\'\n        name: condition-1-child\n        template: condition-1-child\n        when: \'{{inputs.parameters.flip-output}} == heads\'\n  - dag:\n      tasks:\n      - arguments:\n          parameters:\n          - name: flip-again-output\n            value: \'{{tasks.flip-again.outputs.parameters.flip-again-output}}\'\n          - name: flip-output\n            value: \'{{inputs.parameters.flip-output}}\'\n        dependencies:\n        - flip-again\n        name: condition-2\n        template: condition-2\n      - arguments:\n          parame

In [116]:
load_component(filename=yaml)

TypeError: Error: Structure "OrderedDict([('generateName', 'pipeline-flip-coin-')])" is not None.
Error: MetadataSpec.from_dict(struct=OrderedDict([('generateName', 'pipeline-flip-coin-')])) failed with exception:
__init__() got an unexpected keyword argument 'generateName'
Error: Structure "OrderedDict([('generateName', 'pipeline-flip-coin-')])" is incompatible with type "typing.Union[kfp.components._structures.MetadataSpec, NoneType]" - none of the types in Union are compatible.

In [114]:
load_component_from_text(txt)

TypeError: Error: Structure "OrderedDict([('generateName', 'pipeline-flip-coin-')])" is not None.
Error: MetadataSpec.from_dict(struct=OrderedDict([('generateName', 'pipeline-flip-coin-')])) failed with exception:
__init__() got an unexpected keyword argument 'generateName'
Error: Structure "OrderedDict([('generateName', 'pipeline-flip-coin-')])" is incompatible with type "typing.Union[kfp.components._structures.MetadataSpec, NoneType]" - none of the types in Union are compatible.

In [103]:
try_extract_pipeline_zip(bytes)

In [55]:
type(ti)

bytes

In [87]:
ti = try_extract_pipeline_tar(bytes); ti

b'apiVersion: argoproj.io/v1alpha1\nkind: Workflow\nmetadata:\n  generateName: pipeline-flip-coin-\nspec:\n  arguments:\n    parameters: []\n  entrypoint: pipeline-flip-coin\n  serviceAccountName: pipeline-runner\n  templates:\n  - inputs:\n      parameters:\n      - name: flip-output\n    name: condition-1\n    steps:\n    - - arguments:\n          parameters:\n          - name: flip-output\n            value: \'{{inputs.parameters.flip-output}}\'\n        name: condition-1-child\n        template: condition-1-child\n        when: \'{{inputs.parameters.flip-output}} == heads\'\n  - dag:\n      tasks:\n      - arguments:\n          parameters:\n          - name: flip-again-output\n            value: \'{{tasks.flip-again.outputs.parameters.flip-again-output}}\'\n          - name: flip-output\n            value: \'{{inputs.parameters.flip-output}}\'\n        dependencies:\n        - flip-again\n        name: condition-2\n        template: condition-2\n      - arguments:\n          parame

In [ ]:
ti.

In [85]:
b = ti.read(); b

ValueError: I/O operation on closed file

In [77]:
ti.isfile()

True

In [78]:
ti.tobuf()

b'pipeline.yaml\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x000000600\x001003501\x000257523\x0000000017413\x0013371136303\x00015775\x00 0\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00ustar  \x00qimingj\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00primarygroup\x00\x00\x00\x00\x00\x00\x00\x00

In [ ]:
def load_component_from_tar()

In [14]:
is_zipfile(zip)

True

In [3]:
load_component(url=url)

ReaderError: unacceptable character #x008b: invalid start byte
  in "<file>", position 1

In [10]:
load_component(filename=file)

ReaderError: unacceptable character #x008b: invalid start byte
  in "/Users/ryan/c/pipelines/backend/coin.tar.gz", position 1

In [5]:
from zipfile import is_zipfile

In [11]:
is_tarfile(file[:-len('.gz')]), is_tarfile(file)

(True, True)

In [9]:
with open(file[:-len('.gz')], 'rb') as f:
    print(is_zipfile(f))

AttributeError: '_io.BufferedReader' object has no attribute 'startswith'

In [15]:
GET(url)

InvalidSchema: No connection adapters were found for 'gs://ml-pipeline-playground/coin.tar.gz'

In [19]:
import kfp

In [20]:
kfp.__file__

'/Users/ryan/.pyenv/versions/kf-pipelines-py3/lib/python3.5/site-packages/kfp/__init__.py'

In [23]:
url.startswith('gs://')

True